# 전처리

## 모델링을 위해 제공된 데이터들을 2번(교통사고 격자.geojson) 격자에 맞추기로 결정
## Compas Jupyter에서 Server가 계속 끊기는 현상 발생
## -> 외부 Jupyer에서 돌리기 위해 비공개 데이터-2번이 아닌 공개 데이터를 활용하기로 결정 

### 2번 격자 = 30번 격자 동일한지 확인

In [1]:
# 가져다 쓸 모듈들
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # 예쁜 matplotlib
import folium # 지도 시각화
import json # geojson 파일용
import geopandas as gpd # geojson 파일용
import matplotlib.font_manager as fm # 한글 폰트 사용

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
filename = "2.대전광역시_교통사고격자(2017~2019).geojson"
file = open(filename)
df1 = gpd.read_file(file)
df1

,gid,acci_cnt,geometry
0,다바866110,0,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,다바823157,0,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,다바928260,0,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,다바931203,2,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,다바836109,0,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."
...,...,...,...
54907,다바861065,0,"MULTIPOLYGON (((127.34527 36.25565, 127.34527 ..."
54908,다바912036,0,"MULTIPOLYGON (((127.40208 36.22956, 127.40207 ..."
54909,다바867189,0,"MULTIPOLYGON (((127.35174 36.36745, 127.35174 ..."
54910,다바851062,0,"MULTIPOLYGON (((127.33415 36.25293, 127.33415 ..."


In [3]:
filename = "30.대전광역시_차량등록현황_격자.geojson"
file = open(filename)
df2 = gpd.read_file(file)
df2

,gid,cars_cnt,geometry
0,다마846996,0,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,0,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,다마847996,0,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ..."
3,다마847997,0,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ..."
4,다마847998,0,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ..."
...,...,...,...
54907,라바052223,0,"MULTIPOLYGON (((127.55799 36.39818, 127.55799 ..."
54908,라바052224,0,"MULTIPOLYGON (((127.55799 36.39908, 127.55799 ..."
54909,라바053222,0,"MULTIPOLYGON (((127.55910 36.39727, 127.55910 ..."
54910,라바053223,0,"MULTIPOLYGON (((127.55910 36.39818, 127.55910 ..."


In [4]:
(df1['gid'].reset_index(drop=True)==df2['gid'].reset_index(drop=True)).sum() # 54912행 전부 같은 것을 확인

1

## point 형식의 geometry 데이터를 30번 격자에 맞게 변환하기 위하여
## MultiPolygon (격자) 안에 있는 경우를 Check

In [ ]:
# 1. 약 55000개로 이루어진 대구 격자 데이터 로드
filename = "30.대전광역시_차량등록현황_격자.geojson"
file = open(filename,encoding='UTF8')
df2 = gpd.read_file(file)
df2

In [ ]:
# 2. Point 데이터들을 로드, 격자 내에 있는 경우를 Check해서 결과를 json 파일로 저장
filename = ["3.대전광역시_신호등(보행등).geojson","4.대전광역시_신호등(차량등).geojson","7.대전광역시_도로속도표시.geojson","9.대전광역시_교통안전표지.geojson","10.대전광역시_교통CCTV.geojson"]
for i in range(len(filename)):
    file = open(filename[i],encoding='UTF8')
    gc_plz = gpd.read_file(file)
    #print(gc_plz)
    print('==========================================================================================')
    print('==========================현재 진행 상태 : ' + str(i+1) + '번째 file ==================================')
    print(str(len(gc_plz['geometry']))+'번 돌면 끝!')
    print('==========================================================================================')
    gc=[]
    for a in range(len(gc_plz['geometry'])):
        for b in range(len(df2['geometry'])):
            if gc_plz['geometry'][a].within(df2['geometry'][b])==True:
                print(str(a+1)+'번째 index : '+ str(b))
                gc.append(b)  
                break
            if b==len(df2['geometry'])-1 and gc_plz['geometry'][a].within(df2['geometry'][b])==False:
                print(str(a+1)+'번째')
                print('=================================격자가 없어요===============================')
                gc.append(-1)
    gc_plz['index']=pd.DataFrame({'index':gc})
    gc_plz.to_file("3월22일제발마지막-"+str(i+1)+".json", driver="GeoJSON")
    print('=============================================끝==========================================')